In [39]:
import pandas as pd 
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer  
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score



In [17]:
train = pd.read_csv("/Users/iiacymm/Desktop/LoanPrediction/train_u6lujuX_CVtuZ9i.csv")
test = pd.read_csv("/Users/iiacymm/Desktop/LoanPrediction/test_Y3wMUE5_7gLdaTN.csv")

In [18]:
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Gender             356 non-null    object 
 2   Married            367 non-null    object 
 3   Dependents         357 non-null    object 
 4   Education          367 non-null    object 
 5   Self_Employed      344 non-null    object 
 6   ApplicantIncome    367 non-null    int64  
 7   CoapplicantIncome  367 non-null    int64  
 8   LoanAmount         362 non-null    float64
 9   Loan_Amount_Term   361 non-null    float64
 10  Credit_History     338 non-null    float64
 11  Property_Area      367 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


<b> Data Wrangling<b>
- drop: Gender , Married , Education , Self_Employed 
- Dependents: 0 
- Loan_Amount_Term and LoanAmount : KNN Imputer
- Credit_History : IterativeImputer

In [20]:
train.replace(' ', np.nan, inplace = True)
test.replace(' ', np.nan, inplace = True)

In [21]:
train["Dependents"] = train["Dependents"].replace(np.nan ,0) 
test["Dependents"] = test["Dependents"].replace(np.nan ,0) 

In [22]:
train.drop(columns=[ "Gender" , "Married" , "Education" , "Self_Employed"] ,inplace = True)
test.drop(columns=[ "Gender" , "Married" , "Education" , "Self_Employed"] ,inplace = True)

In [23]:
impute_it = IterativeImputer()
credit_imputed = pd.DataFrame(impute_it.fit_transform(train[["Credit_History"]]), columns=["Credit_History"])

In [24]:
impute_it = IterativeImputer()
credit_imputed = pd.DataFrame(impute_it.fit_transform(train[["Credit_History"]]), columns=["Credit_History"])
train["Credit_History"] = credit_imputed

In [25]:
impute_it = IterativeImputer()
credit_imputed = pd.DataFrame(impute_it.fit_transform(test[["Credit_History"]]), columns=["Credit_History"])
test["Credit_History"] = credit_imputed

In [26]:
impute_knn = KNNImputer(n_neighbors = 3)
lAmount_imputed = pd.DataFrame(impute_knn.fit_transform(train[["LoanAmount"]]), columns=["LoanAmount"])
train["LoanAmount"] = lAmount_imputed

In [27]:
impute_knn = KNNImputer(n_neighbors = 3)
lAmount_imputed = pd.DataFrame(impute_knn.fit_transform(test[["LoanAmount"]]), columns=["LoanAmount"])
test["LoanAmount"] = lAmount_imputed

In [28]:
impute_knn = KNNImputer(n_neighbors = 3)
lterm_imputed = pd.DataFrame(impute_knn.fit_transform(train[["Loan_Amount_Term"]]), columns=["Loan_Amount_Term"])
train["Loan_Amount_Term"] = lterm_imputed

In [29]:
impute_knn = KNNImputer(n_neighbors = 3)
lterm_imputed = pd.DataFrame(impute_knn.fit_transform(test[["Loan_Amount_Term"]]), columns=["Loan_Amount_Term"])
test["Loan_Amount_Term"] = lterm_imputed

In [30]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Dependents         614 non-null    object 
 2   ApplicantIncome    614 non-null    int64  
 3   CoapplicantIncome  614 non-null    float64
 4   LoanAmount         614 non-null    float64
 5   Loan_Amount_Term   614 non-null    float64
 6   Credit_History     614 non-null    float64
 7   Property_Area      614 non-null    object 
 8   Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(4)
memory usage: 43.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            367 non-null    object 
 1   Dependents         367 non-null    object 
 2   Appl

In [42]:
X = train[['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']]
Y = train['Loan_Status']
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
lm = LogisticRegression(max_iter=100, random_state=0).fit(X_train, Y_train)

In [44]:
predictions = lm.predict(X_val)
ac = accuracy_score(Y_val,predictions)
print( "Model Evaluation Metrics:")
print("Accuracy:",ac)

Model Evaluation Metrics:
Accuracy: 0.7886178861788617
